In [1]:
import numpy as np
import pandas as pd
import re
import torch
import os
from torch.utils.data import DataLoader, random_split, Dataset
import requests
from datasets import load_dataset
from datasets import Dataset

In [16]:
df = pd.read_csv("hf://datasets/ImruQays/Quran-Classical-Arabic-English-Parallel-texts/Quran-translations.csv")

In [43]:
data = df.copy()

data = data.drop(columns=[data.columns[0]])
data = data[~data['arabic-imlaei'].str.contains("بِسمِ اللَّهِ الرَّحمٰنِ الرَّحيمِ")].reset_index(drop=True)

In [99]:
n = 3118
for i, col in enumerate(data.columns):
    meh = [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    if i not in meh:
        print(f"{i}- {data.loc[n, col]}")

0- إِنَّهُم عَنِ السَّمعِ لَمَعزولونَ
7- They are barred from hearing.


In [100]:
# Old English translations, keep them if the resources are good enough
data.drop(columns=data.columns[meh], inplace=True)

In [107]:
df = pd.DataFrame()
df['en'] = data.iloc[:, 1]
df['ar'] = data.iloc[:, 0]

In [7]:
# # Connect Contextual verses
# connection_signs = ('-', '—', ':', ';', ',')

# temp_df = {
#     "source": [],
#     "target": []
# }

# for column in data.columns[1:]:
#     index = 1
#     src, tgt = data.loc[0, column], data.loc[0, "arabic-imlaei"]
    
#     while index < len(data):

#         if src.endswith(connection_signs):
#             src += f" {data.loc[index, column]}" 
#             tgt += f" {data.loc[index, 'arabic-imlaei']}"
            
#         else:
#             temp_df['source'].append(src)
#             temp_df['target'].append(tgt)
            
#             src, tgt = data.loc[index, column], data.loc[index, "arabic-imlaei"]
            
#         index += 1

In [105]:
def prepare_text(text):
    # Convert text to lowercase
    text = text.lower()

    text = re.sub(r'[\[\(<«](\d+)[\]\)>»]', '', text)
    # Remove text between parentheses/brakets (Explanatory text which is not originally written in arabic)
    text = re.sub(r'\([^)]*\)|\[[^]]*\]', '', text)
    
    # Remove dashes and commas
    # text = re.sub(r'[-;”“’‘\"\'\[\]\(\)﴾﴿<>«»]+', '', text)
    
    # arabic_diacritics = re.compile("""
    #     ٰ    | # الألف الصغيرة
    #     ّ    | # الشدة
    #     َ    | # الفتحة
    #     ً    | # تنوين الفتح
    #     ُ    | # الضمة
    #     ٌ    | # تنوين الضم
    #     ِ    | # الكسرة
    #     ٍ    | # تنوين الكسر
    #     ْ    | # السكون
    #     ۚ    | #
    #     ـ    # 
    # """, re.VERBOSE)
    
    # text = re.sub(arabic_diacritics, '', text)
    
    # text = re.sub(r'\.\.\.', '', text)
    # text = re.sub(r'\*\*\*', '', text)
    
    # Remove البسملة as it is not translated.
    # text = re.sub(r'^بسم الله الرحمن الرحيم ', '', text)
        
    # text = re.sub(r"صلى الله عليه وسلم", '', text)
    # text = re.sub(r"عليه الصلاة والسلام", '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    return text

In [109]:
df = df.map(prepare_text)

In [124]:
df.sample().values

array([['they will be presented before your lord in a row. “you have come to us as we created you the first time. although you claimed we would not set a meeting for you.”',
        'وَعُرِضوا عَلىٰ رَبِّكَ صَفًّا لَقَد جِئتُمونا كَما خَلَقناكُم أَوَّلَ مَرَّةٍ بَل زَعَمتُم أَلَّن نَجعَلَ لَكُم مَوعِدًا']],
      dtype=object)

In [125]:
df.to_csv("Finished.csv")